In [282]:
import pandas as pd

In [283]:
df = pd.read_csv('Cape_July24.csv')


/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_76846/3061799828.py:1: DtypeWarning: Columns (93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Cape_July24.csv')


In [284]:
pd.set_option('display.max_columns', None)
dft = df[['Pitcher','PitcherTeam','RelHeight', 'SpinRate', 'RelSpeed','PitchCall','TaggedPitchType','SpinRate','HorzBreak','InducedVertBreak']]

In [285]:
#dft = dft[dft['Pitcher'] == 'Middleton, JB']
#dft = dft[dft['Batter'] == 'Bodine, Caden']

In [286]:
# First, calculate the total number of swings, which is the sum of swinging strikes and balls hit into play
#total_swings = dft[dft['PitchCall'].isin(['StrikeSwinging','FoulBall','InPlay'])].groupby('TaggedPitchType').size()
#Then calculate the number of whiffs (swinging strikes)
#whiffs = dft[dft['PitchCall'] == 'StrikeSwinging'].groupby('TaggedPitchType').size()
#Finally, calculate the whiff percentage
#whiff_percentages = (whiffs / total_swings) * 100
#whiff_percentages = whiff_percentages.fillna(0)
#whiff_percentages

In [287]:
#pitch type unique values
pitch_type = dft['TaggedPitchType'].unique()
pitch_type

array(['FourSeamFastBall', 'Slider', 'Curveball', 'ChangeUp', 'Splitter',
       'Sinker', 'Fastball', 'TwoSeamFastBall', 'Cutter', 'Undefined',
       'Other', 'Knuckleball', 'OneSeamFastBall'], dtype=object)

In [288]:
#group all fastballs and sinkers together and name them 'Fastball'
dft.loc[dft['TaggedPitchType'].str.contains('Sinker'), 'TaggedPitchType'] = 'Fastball'

dft.loc[dft['TaggedPitchType'].str.contains('FastBall'), 'TaggedPitchType'] = 'Fastball'

dft.loc[dft['TaggedPitchType'].str.contains('FourSeamFastBall'), 'TaggedPitchType'] = 'Fastball'
dft.loc[dft['TaggedPitchType'].str.contains('TwoSeamFastBall'), 'TaggedPitchType'] = 'Fastball'

In [289]:
#create a new column with a value of 1 if the pitch was a whiff, and 0 if it was not
dft['Whiff'] = dft['PitchCall'].apply(lambda x: 1 if x == 'StrikeSwinging' else 0)
#group by pitcher and pitch type, and sum the number of whiffs
whiffs = dft.groupby(['Pitcher','TaggedPitchType'])['Whiff'].sum()
#Count called strikes
called = dft[dft['PitchCall'] == 'StrikeCalled'].groupby(['Pitcher','TaggedPitchType']).size()
#count the total number of pitches thrown
total_pitches = dft.groupby(['Pitcher','TaggedPitchType']).size()
#group by pitcher and pitch type, and count the total number of swings
total_swings = dft[dft['PitchCall'].isin(['StrikeSwinging','FoulBall','InPlay'])].groupby(['Pitcher','TaggedPitchType']).size()
#remove pitchers with less than 50 total swings
total_pitches = total_pitches[total_pitches > 50]
#total_swings = total_swings[total_swings > 20]
#only show fastballs
#total_swings = total_swings[total_swings.index.get_level_values('TaggedPitchType') == 'Slider']

#calculate the whiff percentage
whiff_percentages = (whiffs / total_swings) * 100
whiff_percentages = whiff_percentages.fillna(0)

#calculate the called strike percentage
called_percentages = (called / total_pitches) * 100
called_percentages = called_percentages.fillna(0)


#CSW
csw = (whiffs + called) / total_pitches * 100
csw = csw.fillna(0)






/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_76846/1700099378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft['Whiff'] = dft['PitchCall'].apply(lambda x: 1 if x == 'StrikeSwinging' else 0)


In [290]:
#make a dataframe with the called percentages
dfc = pd.DataFrame(called_percentages)
#rename the column
dfc.columns = ['CalledStrike%']
#reset the index
dfc = dfc.reset_index()


In [291]:
#make a new dataframe with the whiff percentages as a column
dfw = pd.DataFrame(whiff_percentages)
#reset the index
dfw = dfw.reset_index()
#rename the column
dfw = dfw.rename(columns={0:'Whiff%'})
#sort the values
dfw = dfw.sort_values(by='Whiff%', ascending=False)


In [292]:
#create a new dataframe from DF that find the averages of each pitch type by 'SpinRate', 'RelSpeed','TaggedPitchType','SpinRate','HorzBreak','InducedVertBreak'
dfavg = dft.groupby(['Pitcher','PitcherTeam','TaggedPitchType']).mean()
dfavg = dfavg.reset_index()




In [293]:
#combine the two dataframes
dfw = pd.merge(dfw, dfavg, on=['Pitcher','TaggedPitchType'])
#remove spinrate and whiff
dfw = dfw.drop(['SpinRate','Whiff'], axis=1)

#move whiff percentage to the back
dfw = dfw[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','HorzBreak','InducedVertBreak','Whiff%']]

#sort the values



In [294]:
#create new df with total pitches as a column
dfp = pd.DataFrame(total_pitches)
#reset the index
dfp = dfp.reset_index()
#rename the column
dfp = dfp.rename(columns={0:'TotalPitches'})
#sort the values



In [295]:
#combine the two dataframes
f_whiff = pd.merge(dfw, dfp, on=['Pitcher','TaggedPitchType'])


#move whiff percentage to the back
f_whiff = f_whiff[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','HorzBreak','InducedVertBreak','Whiff%','TotalPitches']]


In [296]:
#make a new datafram with csw percentages as a column
dfcsw = pd.DataFrame(csw)
#reset the index
dfcsw = dfcsw.reset_index()
#rename the column
dfcsw = dfcsw.rename(columns={0:'CSW%'})


In [297]:
#merge the two dataframes
f_whiff = pd.merge(f_whiff, dfcsw, on=['Pitcher','TaggedPitchType'])


In [298]:
#merge the called strike percentage
final_whiff = pd.merge(f_whiff, dfc, on=['Pitcher','TaggedPitchType'])
#add a column for CSW
#final_whiff['CSW%'] = final_whiff['Whiff%'] + final_whiff['CalledStrike%']
#move called strike percentage to the back

final_whiff = final_whiff[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','InducedVertBreak','HorzBreak','Whiff%','CalledStrike%','CSW%','TotalPitches']]
#round the values
final_whiff = final_whiff.round(2)
#create a new column for rank






In [299]:
#sort the values
final_whiff = final_whiff.sort_values(by='CSW%', ascending=False)
#final_whiff.head(20)

,Pitcher,PitcherTeam,TaggedPitchType,RelSpeed,RelHeight,InducedVertBreak,HorzBreak,Whiff%,CalledStrike%,CSW%,TotalPitches
0,"Miller, Simon",HYA_HAR,Slider,87.32,5.85,1.66,0.50,65.91,13.16,51.32,76
11,"Matson, Sean",ORL_FIR,Slider,80.10,5.70,0.32,-6.68,46.15,26.32,47.37,57
12,"Picard, Coleman",WAR_GAT,Curveball,79.36,5.89,-16.75,-12.58,46.15,33.33,45.10,51
32,"Quigley, Stephen",FAL_COM,Slider,81.24,5.37,1.07,-12.16,40.00,31.03,44.83,58
96,"Sivley, Kros",HYA_HAR,Slider,76.73,4.79,2.05,14.29,29.41,30.26,43.42,76
6,"Beidelschies, Landon",YAR_RED,Slider,81.69,5.89,0.56,4.27,50.00,22.64,43.40,53
61,"Schlesinger, Rafe",COT_KET,Slider,82.17,4.44,1.56,6.48,34.04,28.18,42.73,110
17,"Clark, Derek",ORL_FIR,Slider,80.33,5.13,1.43,7.70,45.00,21.84,42.53,87
20,"Hill, Camron",COT_KET,Slider,78.15,5.87,4.22,9.82,44.19,20.45,42.05,88
52,"Highfill, Sam",FAL_COM,Fastball,91.08,5.88,17.36,11.18,36.00,27.42,41.94,62


In [300]:
#only show hyannis pitchers
final_whiff = final_whiff[final_whiff['PitcherTeam'] == 'HYA_HAR']
#final_whiff.head(20)

,Pitcher,PitcherTeam,TaggedPitchType,RelSpeed,RelHeight,InducedVertBreak,HorzBreak,Whiff%,CalledStrike%,CSW%,TotalPitches
0,"Miller, Simon",HYA_HAR,Slider,87.32,5.85,1.66,0.50,65.91,13.16,51.32,76
96,"Sivley, Kros",HYA_HAR,Slider,76.73,4.79,2.05,14.29,29.41,30.26,43.42,76
19,"Bates, Ethan",HYA_HAR,Slider,81.66,5.31,5.65,-7.71,44.83,19.35,40.32,62
27,"Obermueller, Cade",HYA_HAR,Slider,81.33,4.17,7.98,15.78,41.94,23.53,38.82,85
72,"Lovasz, Carter",HYA_HAR,Fastball,87.18,4.58,11.06,17.73,32.65,18.18,36.36,88
110,"Trumper, Thaniel",HYA_HAR,Curveball,75.38,6.16,-11.73,-11.87,27.27,30.19,35.85,53
126,"Voelker, Zach",HYA_HAR,Fastball,90.36,5.81,16.61,14.28,25.00,22.95,34.43,61
14,"Horn, Darin",HYA_HAR,Slider,84.36,5.28,-1.42,-9.48,45.83,18.57,34.29,70
138,"Arnold, Jamie",HYA_HAR,Fastball,89.66,3.96,2.95,-19.64,24.00,25.25,34.16,202
29,"Lanthier, Ethan",HYA_HAR,Slider,78.81,5.90,1.27,-19.30,41.67,21.95,34.15,82
